In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Input, Activation, Dropout, Bidirectional
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers.embeddings import Embedding
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.preprocessing import sequence
from keras import backend as K
from keras.models import Model

from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf


import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [2]:
def report(le, y_test, y_pred):
        print("Classifation Report")

        target_names = le.classes_
        class_indices = {cls: idx for idx, cls in enumerate(le.classes_)}
        print(class_indices)

        print(metrics.classification_report(y_test, y_pred, target_names=target_names,
                                            labels=["" + str(class_indices[cls]) for cls in target_names]))

        print("============================================================")
        print("Confusion matrix")
        print("============================================================")
        print(target_names)
        print(confusion_matrix(y_test, y_pred,
            labels=[class_indices[cls] for cls in target_names]))

        precision_micro, recall_micro, fscore_micro, _ = \
        precision_recall_fscore_support(y_test, y_pred, average='micro', pos_label=None)

        precisions_macro, recalls_macro, fscore_macro, _ = \
        precision_recall_fscore_support(y_test, y_pred, average='macro', pos_label=None)

        precisions_weighted, recalls_weighted, fscore_weighted, _ =\
        precision_recall_fscore_support(y_test, y_pred, average='weighted', pos_label=None)

        measures = "p: %.4f r: %.4f f1: %.4f"
        print("Micro:  " + measures % (precision_micro, recall_micro, fscore_micro))
        print("Macro:  " + measures % (precisions_macro, recalls_macro, fscore_macro))
        print("Weight: " + measures % (precisions_weighted, recalls_weighted, fscore_weighted))

        print('Test Accuracy: %.4f' % accuracy_score(y_test, y_pred))
        #print('ROC AUC: %.3f' % roc_auc_score(y_true=y_test, y_score=y_pred))
        print("============================================================")


In [3]:
import os
import re
#from keras.preprocessing.text import Tokenizer
import numpy as np


def get_texts_from_dir(cat_dir):
    texts = []
    f_names = []
    DIR = cat_dir

    for f_name in sorted(os.listdir(DIR)):
        f_path = os.path.join(DIR, f_name)
        f = open(f_path, "r")
        texts += [f.read()]
        f.close()
    print("%d files loaded from %s" % (len(texts), cat_dir))
    return texts

def get_texts_from_dir2(cat_dir, truth_file_path):
    texts = []
    f_names = []
    DIR = cat_dir
    f_truth = open(truth_file_path)
    dict_truth = {}
    for line in f_truth:
        line = line.strip().split(" ")
        dict_truth[line[0]] = (line[1])
    f_truth.close()
    
    categories = []
    for f_name in sorted(os.listdir(DIR)):
        f_path = os.path.join(DIR, f_name)
        f = open(f_path, "r")
        texts += [f.read()]
        f_names += [f_name]
        categories += [dict_truth[f_name[:-4]]]
        f.close()
    print("%d files loaded from %s" % (len(texts), cat_dir))
    return f_names, texts, categories

In [4]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
tr_txt = get_texts_from_dir("./2017_txt/positive/")
tr_txt2 = get_texts_from_dir("./2017_txt/negative/")

f_names, X_test, y_test = get_texts_from_dir2("./2017_txt/test/","./test_golden_truth2017.txt")

f_names2, X_test_2019, y_test_2019 = get_texts_from_dir2("./2018_txt/","./test_golden_truth2018.txt")

n_cl1 = len(tr_txt)
n_cl2 = len(tr_txt2)

labpos = []
labneg = []

for i in range(n_cl1):
    labpos.append(1)
for i in range(n_cl2):
    labneg.append(0)
print(type(tr_txt),type(tr_txt2))

#print(f2_names)
#X_train = tr_txt + tr_txt2
X_train = tr_txt + tr_txt2
#y_train = np.concatenate((labpos, labneg), axis=0)
y_train = np.concatenate((labpos, labneg), axis=0)

83 files loaded from ./2017_txt/positive/
403 files loaded from ./2017_txt/negative/
401 files loaded from ./2017_txt/test/
820 files loaded from ./2018_txt/
<class 'list'> <class 'list'>


In [6]:
print(len(X_train[0]))

128413


In [7]:
from keras.preprocessing.text import Tokenizer

# Build Tokenizer and Vocabulary
tokenizer = Tokenizer(num_words=2100, filters="", lower=True)
tokenizer.fit_on_texts(X_train)
# Dictionary of the WHOLE extracted vocabulary
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# Frequencies of words
word_counts = tokenizer.word_counts
print("Frequency of \"%s\" is %s" % ("hello", word_counts["hello"]))

tokenizer.fit_on_texts(X_test)
# Dictionary of the WHOLE extracted vocabulary
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# Frequencies of words
word_counts = tokenizer.word_counts
print("Frequency of \"%s\" is %s" % ("hello", word_counts["hello"]))

tokenizer.fit_on_texts(X_test_2019)
# Dictionary of the WHOLE extracted vocabulary
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# Frequencies of words
word_counts = tokenizer.word_counts
print("Frequency of \"%s\" is %s" % ("hello", word_counts["hello"]))

Found 437587 unique tokens.
Frequency of "hello" is 185
Found 703550 unique tokens.
Frequency of "hello" is 380
Found 1294563 unique tokens.
Frequency of "hello" is 982


In [8]:
inverted_word_index = {}
for word in word_index.keys():
    inverted_word_index[word_index[word]]=word
    
print(inverted_word_index[20])
top_words = len(inverted_word_index)
print(len(inverted_word_index))

max_features = len(inverted_word_index)

have
1294563


In [9]:
# integer encode documents
from keras.preprocessing.text import text_to_word_sequence
from tqdm import tqdm

X_train1 = []
result = []
for i,doc in tqdm(enumerate(X_train)):
    words = list(text_to_word_sequence(doc))
    for w in words:
        if w in word_index:
            result.append(word_index[w])
        else:
            c = 0
    X_train1.append(result)
    result = []
    
print(len(max(X_train1)))
print(len(X_train1))

X_test1 = []
result = []
for i,doc in tqdm(enumerate(X_test)):
    words = list(text_to_word_sequence(doc))
    for w in words:
        if w in word_index:
            result.append(word_index[w])
        else:
            c = 0
    X_test1.append(result)
    result = []
    
print(len(max(X_test1)))
print(len(X_test1))

##
X_test1_2019 = []
result = []
for i,doc in tqdm(enumerate(X_test_2019)):
    words = list(text_to_word_sequence(doc))
    for w in words:
        if w in word_index:
            result.append(word_index[w])
        else:
            c = 0
    X_test1_2019.append(result)
    result = []
    
print(len(max(X_test1_2019)))
print(len(X_test1))

486it [00:04, 84.62it/s] 
9it [00:00, 83.67it/s]

1432
486


401it [00:03, 115.86it/s]
8it [00:00, 75.03it/s]

39503
401


820it [00:08, 99.29it/s] 

145
401


In [10]:
y_test = list(map(int, y_test))

print(type(y_train[0]))
print(type(y_test[0]))

##
y_test_2019 = list(map(int, y_test_2019))

print(type(y_train[0]))
print(type(y_test[0]))

<class 'numpy.int64'>
<class 'int'>
<class 'numpy.int64'>
<class 'int'>


In [11]:
from itertools import islice
from collections import deque

def sliding_window(iterable, size=25, step=1, fillvalue=0): #list, size of window, step, fill empty value
    if size < 0 or step < 1: raise ValueError
    it = iter(iterable)
    q = deque(islice(it, size), maxlen=size)
    if not q: return  # empty iterable or size == 0
    q.extend(fillvalue for _ in range(size - len(q)))  # pad to size
    while True:
        yield iter(q)  # iter() to avoid accidental outside modifications
        q.append(next(it))
        q.extend(next(it, fillvalue) for _ in range(step - 1))

In [12]:
X_train2 = []
y_train2 = []

sequence_len = 35

for X_array,y in tqdm(zip(X_train1,y_train)):
    #print(X_array)
    window = sliding_window(X_array,size=sequence_len,step=sequence_len)

    for i, w in (enumerate(window)):
        l = list(w)
        l = np.asarray(l)
        #print(l)
        X_train2.append(l)
        y_train2.append(int(y))
X_train2 = np.asarray(X_train2)        
print(len(X_train2))
print(len(y_train2))

0it [00:00, ?it/s]/home/hermit/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: generator 'sliding_window' raised StopIteration
  # Remove the CWD from sys.path while we load stuff.
486it [00:02, 193.09it/s]


252110
252110


In [13]:
X_test2 = []
y_test2 = []
for X_array,y in tqdm(zip(X_test1,y_test)):
    #print(X_array)
    window = sliding_window(X_array,size=sequence_len,step=sequence_len)

    for i, w in (enumerate(window)):
        l = list(w)
        l = np.asarray(l)
        #print(l)
        X_test2.append(l)
        y_test2.append(int(y))
X_test2 = np.asarray(X_test2)        
print(len(X_test2))
print(len(y_test2))

0it [00:00, ?it/s]/home/hermit/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: generator 'sliding_window' raised StopIteration
  import sys
401it [00:02, 201.16it/s]


217606
217606


In [14]:
print(type(X_train2[0]), X_train2)

<class 'numpy.ndarray'> [[    1   145   582 ... 28556    14     1]
 [73604  5115 13775 ...     3  3326    14]
 [17670    66    20 ...   440    13    55]
 ...
 [   14   132  3322 ... 25044    71 89174]
 [  362 67462     1 ...  2046   748     3]
 [ 5025   437    83 ...     0     0     0]]


In [16]:
import pickle
from gensim.models import KeyedVectors
from gensim.models import FastText as fText

# Creating the model
embedding_model = KeyedVectors.load_word2vec_format('word2vec-vectors-300.bin',binary=True)

#with open('wiki.en', 'rb') as f:
    #embedding_model = pickle.load(f)
#print(embedding_model["hello"])

#embedding_model = KeyedVectors.load_word2vec_format('GNvectors.bin', binary=True)
embedding_weights = {key: embedding_model[word] if word in embedding_model else
                          np.random.uniform(-0.25, 0.25, 300)
                     for key, word in inverted_word_index.items()}


In [17]:
# create the model
embedding_vecor_length = 300
weights = np.array([embedding_weights[i+1] for i in range(len(embedding_weights))])
class_weight = {0: 1.,
                1: 10.}

#print(weights.shape)

model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=None, trainable=False, weights=[weights]))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         388368900 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               320800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 388,689,901
Trainable params: 321,001
Non-trainable params: 388,368,900
_________________________________________________________________


In [18]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1731532718178137627
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10663444480
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10309501553859979247
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:07:00.0, compute capability: 7.5"
]


In [19]:
model.fit(X_train2, y_train2, validation_data=(X_test2, y_test2), epochs=15, batch_size=250,class_weight=class_weight)

Train on 252110 samples, validate on 217606 samples
Epoch 1/15
252110/252110 [==============================] - 105s 416us/step - loss: 1.2945 - acc: 0.5042 - val_loss: 0.7836 - val_acc: 0.4757
Epoch 2/15
252110/252110 [==============================] - 103s 409us/step - loss: 1.2416 - acc: 0.5413 - val_loss: 0.8091 - val_acc: 0.4929
Epoch 3/15
252110/252110 [==============================] - 104s 412us/step - loss: 1.1823 - acc: 0.5795 - val_loss: 0.7805 - val_acc: 0.5271
Epoch 4/15
252110/252110 [==============================] - 103s 410us/step - loss: 1.1181 - acc: 0.6131 - val_loss: 0.7324 - val_acc: 0.5770
Epoch 5/15
252110/252110 [==============================] - 103s 409us/step - loss: 1.0614 - acc: 0.6395 - val_loss: 0.6723 - val_acc: 0.6229
Epoch 6/15
252110/252110 [==============================] - 104s 411us/step - loss: 1.0052 - acc: 0.6642 - val_loss: 0.7516 - val_acc: 0.5852
Epoch 7/15
252110/252110 [==============================] - 103s 409us/step - loss: 0.9460 - acc

In [20]:

y_prediction = []
def most_frequent(List): 
    return max(set(List), key = List.count) 

for X_array in tqdm(X_test1_2019):
    #print(X_array)
    window = sliding_window(X_array,size=sequence_len,step=sequence_len)
    
    X_test3 = []
    for i, w in (enumerate(window)):
        l = list(w)
        l = np.asarray(l)
        #print(l)
        X_test3.append(l)   
        
    X_test3 = np.asarray(X_test3)
    predictions = model.predict(X_test3, verbose=0)
    
    results = []
    for j in predictions:
        if j >= 0.5:
            results.append(1)
        else:
            results.append(0)
    
    y_p = most_frequent(results)
    y_prediction.append(y_p)
print(len(y_prediction),len(y_test_2019))
#print(len(X_test2))
#print(len(y_test2))

#predictions = model.predict(X_test1, verbose=0)
#print(predictions)

  0%|          | 0/820 [00:00<?, ?it/s]/home/hermit/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: generator 'sliding_window' raised StopIteration
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 820/820 [06:14<00:00,  2.19it/s]

820 820


In [21]:
print(y_prediction)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics, preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score

label_encoder = LabelEncoder()

y_te = label_encoder.fit_transform(" ".join(map(str, y_test_2019)).split())
report(label_encoder, y_te, y_prediction)



Classifation Report
{'0': 0, '1': 1}
             precision    recall  f1-score   support

          0       0.96      0.90      0.93       741
          1       0.40      0.61      0.48        79

avg / total       0.90      0.88      0.89       820

Confusion matrix
['0' '1']
[[670  71]
 [ 31  48]]
Micro:  p: 0.8756 r: 0.8756 f1: 0.8756
Macro:  p: 0.6796 r: 0.7559 f1: 0.7071
Weight: p: 0.9026 r: 0.8756 f1: 0.8864
Test Accuracy: 0.8756


/home/hermit/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
